## Categorical Encoding

1. Using sklearn libraries
2. Using category_encoders - more user friendly

In [263]:
import os
import pandas as pd
import missingno as msno
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper, CategoricalImputer
import numpy as np
from sklearn.compose import ColumnTransformer 
import category_encoders as ce

os.chdir('/Users/suma/Documents/01 Data Science/Titanic Problem/')

In [264]:
df_train = pd.read_csv('titanic_train.csv')
df_test = pd.read_csv('titanic_test.csv')

#### Append Train and Test data (for preprocessing)

In [294]:
frames = [df_train, df_test]
df = pd.concat(frames, axis = 0, sort = False)
#df.info()

#### Possible Errors
1) When Input field is integer or numerical and you are applying encoding Examaple:PClass <br>
2) When there are null values in input fields Example:Embarked <br>
3) For label encoding - 1st unique value is mapped to 1, 2nd unique value is mapped to 2 and so on. So for example, unique values are First, Third, Fourth, Tenth, Second... they would be encoded 1, 2, 3, 4, 5 ... This mapping would be wrong for ordinal data. 

#### Label Encoding (sklearn libraries)

In [ ]:
le = preprocessing.LabelEncoder()
#label encoders work on series data, but not on frames
dummy = ['A', 'B', 'C', 'A', 'C']
le.fit(dummy)
le.classes_
le.transform(dummy)

le.fit(df['Sex'])
print(le.classes_)
print(le.transform(df['Sex']))

cat_imputer = CategoricalImputer()
df['Embarked'] = cat_imputer.fit_transform(df['Embarked'])

df[encodable_features] = df[encodable_features].apply(le.fit_transform)

#### One Hot Encoding (sklearn libraries)
##### Method 1: Suitable for complex pipelines

In [250]:
encodable_features = ['Pclass','Sex','Embarked']

cat_imputer = CategoricalImputer()
df['Embarked'] = cat_imputer.fit_transform(df['Embarked'])

ct = ColumnTransformer([("OneHotEncode", preprocessing.OneHotEncoder(),encodable_features)], remainder="drop") # The last arg ([0]) is the list of columns you want to transform in this step
one_hot_encode_columns = ct.fit_transform(df)
new_col_names = []
for col in encodable_features:
    for s in df[col].unique():
        new_col_names.append(col +'_'+ str(s))
df_1hot = pd.DataFrame(data = one_hot_encode_columns, columns = new_col_names)
df[new_col_names] = df_1hot

In [251]:
df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived,Ticket,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female,Embarked_S,Embarked_C,Embarked_Q
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S,0.0,A/5 21171,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C,1.0,PC 17599,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S,1.0,STON/O2. 3101282,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S,1.0,113803,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S,0.0,373450,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


##### Method2: Suitable for quick model build

In [257]:
df = pd.get_dummies(df, columns = encodable_features)

#### Category Encoders

##### 1. Ordinal Encoder (Label Encoder)

Sklearn’s LabelEncoder does pretty much the same thing as Category Encoder’s OrdinalEncoder, but is not quite as user friendly. LabelEncoder won’t return a DataFrame, instead it returns a numpy array if you pass a DataFrame. It also outputs values starting with 0, compared to OrdinalEncoder’s default of outputting values starting with 1.

You could accomplish ordinal encoding by mapping string values to integers manually with apply. But that’s extra work once you know how to use Category Encoders.

In [272]:
ce_ord = ce.OrdinalEncoder(cols = 'Pclass')
pclass1 = ce_ord.fit_transform(df['Pclass'])

In [275]:
#The above statement encodes as follows 3 -> 1, 1 -> 2, 2 -> 3, based on first occurance in df. To avoid this follow the below
dict = [{'col': 'Pclass', 
  'mapping': [('1', 1), 
              ('2', 2), 
              ('3', 3)]}]

ce_ord = ce.OrdinalEncoder(dict)
pclass2 = ce_ord.fit_transform(df['Pclass'])

##### 2. OneHot Encoder

In [279]:
ce_one_hot = ce.OneHotEncoder(cols = ['Sex','Embarked'])
df = ce_one_hot.fit_transform(df)
#If a column has null values, then new one hot encoded column would be created for Nulls

##### 3. Binary Encoder <br>
Binary can be thought of as a hybrid of one-hot and hashing encoders. Binary creates fewer features than one-hot, while preserving some uniqueness of values in the the column. It can work well with higher dimensionality ordinal data. <br>

- The categories are encoded by OrdinalEncoder if they aren’t already in numeric form. <br>
- Then those integers are converted into binary code, so for example 5 becomes 101 and 10 becomes 1010 <br>
- Then the digits from that binary string are split into separate columns. So if there are 4–7 values in an ordinal column then 3 new columns are created: one for the first bit, one for the second, and one for the third. <br>
- Each observation is encoded across the columns in its binary form.

In [282]:
ce_binary = ce.BinaryEncoder(cols = 'Cabin')
df = ce_binary.fit_transform(df)

##### 4. BaseN Encoder <br>

When the BaseN base = 1 it is basically the same as one hot encoding. When base = 2 it is basically the same as binary encoding. McGinnis said, “Practically, this adds very little new functionality, rarely do people use base-3 or base-8 or any base other than ordinal or binary in real problems.”

The main reason for its existence is to possibly make grid searching easier. You could use BaseN with gridsearchCV. However, if you’re going to grid search with some of these encoding options, you’re going to make that search part of your workflow anyway.

In [288]:
ce_basen = ce.BaseNEncoder(cols = 'Cabin', base = 3)
df = ce_basen.fit_transform(df)

##### 5. Hashing <br>

HashingEncoder implements the hashing trick. It is similar to one-hot encoding but with fewer new dimensions and some info loss due to collisions. The collisions do not significantly affect performance unless there is a great deal of overlap. 

https://booking.ai/dont-be-tricked-by-the-hashing-trick-192a6aae3087

The n_components parameter controls the number of expanded columns. The default is eight columns. In our example column with three values the default results in five columns full of 0s.

If you set n_components less than k you’ll have a small reduction in the value provided by the encoded data. You’ll also have fewer dimensions.

You can pass a hashing algorithm of your choice to HashingEncoder; the default is md5. Hashing algorithms have been very successful in some Kaggle competitions. It’s worth trying HashingEncoder for nominal and ordinal data if you have high cardinality features.

In [292]:
ce_hash = ce.HashingEncoder(cols = 'Cabin')
df = ce_hash.fit_transform(df)

##### 6. Target Encoding / Mean Encoding <br>

Target encoding is the process of replacing a categorical value with the mean of the target variable.

For the case of categorical target: features are replaced with a blend of posterior probability of the target given particular categorical value and the prior probability of the target over all the training data.

For the case of continuous target: features are replaced with a blend of the expected value of the target given particular categorical value and the expected value of the target over all the training data.

In case of large number of features, mean encoding could prove to be a much simpler alternative

Even though it looks like mean encoding is Superman…… it’s kryptonite is overfitting. The fact that we use target classes to encode for our training labels may leak data about the predictions causing the encoding to become biased. Well we can avoid this by Regularizing. Several Kaggle Competitors use mean encoding with regularization to predict much better and rise through ranks in the leaderboard.

In [296]:
ce_target = ce.TargetEncoder(cols = 'Embarked')
df = ce_target.fit_transform(X = df_train, y = df_train['Survived'])

##### 7. Leave one out encoding

This is very similar to target encoding but excludes the current row’s target when calculating the mean target for a level to reduce the effect of outliers.

In [299]:
ce_leave1out = ce.LeaveOneOutEncoder(cols = 'Cabin')
df = ce_leave1out.fit_transform(X = df_train, y = df_train['Survived'])